In [73]:
import glob
# This file should run basically insta fast. Bad coding for sure 
genSavePath = "F:/PythonProjects/NietoExcercise-1/SavedResults/"
day = 29  # Which month and days result you want to visualize in table
month = 10
repetitionValue = 21
repetitionName = "2"
repetitionString = f"{repetitionValue}{repetitionName}"
curSavePath = f"{genSavePath}{day}-{month}-{repetitionString}/"
savedResults = glob.glob(curSavePath + "savedBestSeed*")

skipSubjects = []
onlySignificant = True
minAcc = 0.5
print(len(savedResults))
print(len(savedResults)/9)

90
10.0


In [74]:
import pandas as pd
from array import ArrayType
from requests import delete
from tabulate import tabulate
from prettytable import PrettyTable as pt
import numpy as np


nameDict = dict()
nrOfNames = 0
kernelDict = dict()
nrOfKernel = 0
nrOfCombination = 0
currSeed = 0
currSubj = 0
combinationList = []
combinationResult = []


results = []
for path in savedResults:
    results.append(np.load(path, allow_pickle=True))

subjectAmount = 10
for x in range(subjectAmount):
    combinationResult.append([])

nrOfSeedsPerSubject = np.zeros(subjectAmount)
for s, subject in enumerate(results):
    currSubj = 0
    currSeed = 0
    subjectBestArray = np.empty([len(subject)], dtype=object)

    currSeed = subject[0]
    currSubj = subject[1]

    if currSubj in skipSubjects:
        continue
    nrOfSeedsPerSubject[currSubj] += 1

    for aResults in subject[2]:
        for aResults2 in aResults:
            # If the np array comes from svm results
            if len(aResults2) < 6:

                aName = aResults2[0]
                ares = aResults2[1]

                if type(ares) is tuple:
                    ares = ares[0]

                akernel = aResults2[2]
                aC = round(aResults2[3], 2)

                if aName not in nameDict:
                    nameDict[aName] = nrOfNames
                    nrOfNames += 1
                if akernel not in kernelDict:
                    kernelDict[akernel] = nrOfKernel
                    nrOfKernel += 1
                if [nameDict[aName], kernelDict[akernel], aC] not in combinationList:
                    combinationList.append([nameDict[aName], kernelDict[akernel], aC])
                    nrOfCombination += 1

                combNr = combinationList.index(
                    [nameDict[aName], kernelDict[akernel], aC]
                )
                if any(combNr in sublist for sublist in combinationResult[currSubj]):
                    sublist = np.array(combinationResult[currSubj])
                    index = int(np.where(sublist[:, 0] == combNr)[0])

                    combinationResult[currSubj][index][1] = combinationResult[currSubj][
                        index
                    ][1] * (1 - (1 / nrOfSeedsPerSubject[currSubj])) + ares * (
                        1 / nrOfSeedsPerSubject[currSubj]
                    )

                else:

                    combinationResult[currSubj].append([combNr, ares])

                    # If the np array comes from NN results
            else:

                aName = aResults2[0]
                ares = aResults2[1]

                if type(ares) is tuple:
                    ares = ares[0]
                akernel = aResults2[2]
                aLz = round(aResults2[3], 2)
                areg = round(aResults2[4], 2)
                adrp = round(aResults2[5], 2)
                if aName not in nameDict:
                    nameDict[aName] = nrOfNames
                    nrOfNames += 1
                if akernel not in kernelDict:
                    kernelDict[akernel] = nrOfKernel
                    nrOfKernel += 1
                if [
                    nameDict[aName],
                    kernelDict[akernel],
                    aLz,
                    areg,
                    adrp,
                ] not in combinationList:
                    combinationList.append(
                        [nameDict[aName], kernelDict[akernel], aLz, areg, adrp]
                    )
                    nrOfCombination += 1
                # combResult = []
                combNr = combinationList.index(
                    [nameDict[aName], kernelDict[akernel], aLz, areg, adrp]
                )
                if any(combNr in sublist for sublist in combinationResult[currSubj]):
                    sublist = np.array(combinationResult[currSubj])

                    index = int(np.where(sublist[:, 0] == combNr)[0])

                    combinationResult[currSubj][index][1] = combinationResult[currSubj][
                        index
                    ][1] * (1 - (1 / nrOfSeedsPerSubject[currSubj])) + ares * (
                        1 / nrOfSeedsPerSubject[currSubj]
                    )

                else:

                    combinationResult[currSubj].append([combNr, ares])


# Making table

# headers = [f"{x[0]}:{x[1]}:{x[2]}:" for x in combinationList]
headers = np.arange(nrOfCombination)
subjectResultArray = np.empty([10, nrOfCombination], dtype=object)
subjectNrList = []

comboAverage = np.zeros([1, nrOfCombination])
comboTotal = np.zeros([1, nrOfCombination])
nrOfSubjects = np.zeros(nrOfCombination)

topTen = []
lowestTopRes = [0, 0]
for subNr, cResult in enumerate(combinationResult):
    resArray = np.zeros([nrOfCombination])
    if len(cResult) != 0:
        for res in cResult:
            if onlySignificant == True:
                if res[1] < minAcc:
                    continue
            comboAverage[0, res[0]] = comboAverage[0, res[0]] + res[1]
            comboTotal[0, res[0]] = comboTotal[0, res[0]] + res[1]
            nrOfSubjects[res[0]] += 1
            subjectResultArray[subNr, res[0]] = res[1]

            if len(topTen) < 10 or topTen[0][0] < res[1]:

                if len(topTen) == 10:
                    topTen.reverse()
                    topTen.pop()

                dataF = list(nameDict.keys())[combinationList[res[0]][0]]
                kernel = list(kernelDict.keys())[combinationList[res[0]][1]]
                cVal = combinationList[res[0]][2]
                topTen.append([res[1], subNr, dataF, kernel, cVal])
                topTen.sort()


# Print the table!
comboAverage = comboAverage / nrOfSubjects
print(comboAverage.shape)
print(subjectResultArray.shape)
pd.set_option("display.max_columns", None)

# For loop through zip of headers and columns,
# If column has all values == None, Remove column and header.
subjectResultArray = np.array(subjectResultArray)
emptyCols = []
atleastOneEmpty = False
for hcol, in zip(headers):
    if all(subjectResultArray[:,hcol] == None):
        atleastOneEmpty = True
        emptyCols.append(hcol)
if atleastOneEmpty: 
    subjectResultArray = np.delete(subjectResultArray, np.array(emptyCols),1)
    headers = np.delete(headers, np.array(emptyCols),0)
    comboAverage = np.delete(comboAverage, np.array(emptyCols),1)
    comboTotal = np.delete(comboTotal, np.array(emptyCols),1)
    nrOfSubjects = np.delete(nrOfSubjects, np.array(emptyCols),0)


table = pd.DataFrame(subjectResultArray, columns=headers)
display(table)




(1, 27)
(10, 27)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,None,None,None,None,None,None,0.535,0.51,0.595,None,0.5,None,0.54,0.53,0.505,0.61,0.665,0.595,0.54,0.58,0.665,0.555,0.565,0.63,None,None,None
2,None,None,None,0.575,0.516667,None,0.570833,0.520833,None,None,0.508333,None,None,None,None,None,None,None,None,None,None,None,None,None,0.5125,0.529167,None
3,None,None,None,0.522222,0.538889,0.5,0.588889,None,None,0.538889,None,0.588889,0.566667,None,None,0.516667,0.511111,0.5,None,None,None,None,None,None,None,None,None
4,None,0.558333,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0.5375,None,None,None,0.504167,None
5,None,None,None,0.516667,None,None,None,None,None,0.508333,None,0.583333,None,None,None,0.529167,0.504167,None,0.5375,0.508333,None,0.525,None,None,None,None,None
6,None,None,None,None,None,None,None,None,None,None,None,None,0.545455,0.540909,None,None,None,None,None,0.513636,None,None,0.545455,None,0.540909,None,0.518182
7,None,None,None,0.554167,0.541667,0.5375,None,None,None,0.5,0.5625,0.504167,None,0.541667,0.541667,0.533333,None,0.558333,0.529167,0.566667,0.558333,0.508333,0.583333,0.545833,None,None,None
8,0.52,0.535,0.57,0.635,0.555,0.55,0.645,0.57,0.53,None,None,None,None,None,0.525,None,None,0.55,0.505,0.53,None,None,None,None,0.5,None,None
9,None,None,None,0.55,0.6,0.504167,0.5125,0.516667,0.529167,None,None,None,None,None,0.520833,None,0.5,None,None,None,None,None,None,None,0.570833,0.558333,0.508333


In [75]:
print("Top ten highest accuracies")

topTen.reverse()
for top in topTen:
    print(
            f"Accuracy {round(top[0],2)} : subject {top[1]}, feature:{top[2]}, kernel{top[3]}, C:{top[4]}"
        )


Top ten highest accuracies
Accuracy 0.66 : subject 1, feature:-dataHRCV, kernelsigmoid, C:2.5
Accuracy 0.66 : subject 1, feature:-dataCV, kernelrbf, C:2.5
Accuracy 0.64 : subject 8, feature:-dataHR, kernellinear, C:0.5
Accuracy 0.64 : subject 8, feature:-welchData, kernellinear, C:0.5
Accuracy 0.63 : subject 1, feature:-dataCV2, kernelrbf, C:2.5
Accuracy 0.61 : subject 1, feature:-dataHRCV, kernellinear, C:0.5
Accuracy 0.6 : subject 9, feature:-welchData, kernelsigmoid, C:2.5
Accuracy 0.6 : subject 1, feature:-dataHRCV, kernelrbf, C:2.5
Accuracy 0.6 : subject 1, feature:-dataHR, kernelrbf, C:2.5
Accuracy 0.59 : subject 3, feature:-dataHR, kernellinear, C:0.5


In [76]:
subjAv = np.zeros(subjectResultArray.shape[0])
for s, subj in enumerate(subjectResultArray):
    subjAv[s] = 0
    if all(subj == None):
        continue
    subjAv[s] = np.mean(subj[np.where(subj != None)])
print(subjAv)
pd.set_option("display.max_columns", None)
table = pd.DataFrame(np.array(comboAverage), columns=headers)
display(table)


print(
    f"Max average accuracy {np.max(np.array(comboAverage[np.where(np.isnan(comboAverage)==False)]))}"
)
maxVal = np.max(np.array(comboAverage[np.where(np.isnan(comboAverage) == False)]))
maxInd = int(np.where(comboAverage == maxVal)[1][0])
print(headers[maxInd])
print(nrOfSubjects[maxInd])
print(list(nameDict.keys())[combinationList[maxInd][0]])
print(list(kernelDict.keys())[combinationList[maxInd][1]])
print(f"C: {combinationList[maxInd][2]}")


[0.         0.57       0.53333333 0.53722222 0.53333333 0.5265625
 0.53409091 0.54166667 0.55142857 0.53371212]


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26
0,0.52,0.546667,0.57,0.558843,0.550444,0.522917,0.570444,0.529375,0.551389,0.515741,0.523611,0.558796,0.550707,0.537525,0.523125,0.547292,0.545069,0.550833,0.527917,0.539727,0.611667,0.531458,0.564596,0.587917,0.531061,0.530556,0.513258


Max average accuracy 0.6116666666666666
20
2.0
-dataCV
rbf
C: 2.5


In [77]:
table = pd.DataFrame(np.array(comboTotal), columns=headers)
display(table)

maxVal2 = np.max(np.array(comboTotal[np.where(np.isnan(comboTotal) == False)]))

for maxInd2 in np.where(comboTotal == maxVal2)[1]:

    # maxInd2 = int(np.where(comboTotal == maxVal2)[1])
    print(headers[maxInd2])
    print(nrOfSubjects[maxInd2])
    print(
        f"Max total significant accuracy then averaged {maxVal2/nrOfSubjects[maxInd2]} "
    )

    print(list(nameDict.keys())[combinationList[maxInd2][0]])
    print(list(kernelDict.keys())[combinationList[maxInd2][1]])
    print(f"C: {combinationList[maxInd2][2]} ")


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26
0,0.52,1.093333,0.57,3.353056,2.752222,2.091667,2.852222,2.1175,1.654167,1.547222,1.570833,1.676389,1.652121,1.612576,2.0925,2.189167,2.180278,2.203333,2.111667,2.698636,1.223333,2.125833,1.693788,1.175833,2.124242,1.591667,1.026515


3
6.0
Max total significant accuracy then averaged 0.5588425925925926 
-welchData
linear
C: 0.5 


In [78]:
# maxAcc = np.max([res for res in np.array(
#     subjectResultArray[subjectResultArray != None])])
# print(f"Max single accuracy position {np.where(subjectResultArray == maxAcc)}")
# print(np.max([res for res in np.array(
#     subjectResultArray[subjectResultArray != None])]))
